In [20]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, Scale, HORIZONTAL, Toplevel
from PIL import Image, ImageTk, ImageChops, ImageEnhance, ImageDraw, ImageFont, ImageOps
import io
import platform # Pentru detectia OS (mouse wheel difera pe Linux)

In [21]:
# ==========================================
# CLASA LASSO (NESCHIMBATA)
# ==========================================
class LassoSelectorApp:
    def __init__(self, master_root):
        self.top = Toplevel(master_root)
        self.top.title("Unealtă Lasso")
        self.top.geometry("900x700")
        self.top.grab_set()

        self.source_image = None
        self.tk_source = None
        self.result_image = None
        self.points = []

        btn_frame = tk.Frame(self.top, bg="#2c3e50", pady=5)
        btn_frame.pack(fill=tk.X)
        tk.Button(btn_frame, text="1. Încarcă Sursă", command=self.load_image, bg="#3498db", fg="white").pack(side=tk.LEFT, padx=5)
        tk.Button(btn_frame, text="2. Reset", command=self.reset_selection, bg="#e74c3c", fg="white").pack(side=tk.LEFT, padx=5)
        tk.Button(btn_frame, text="3. DECUPEAZĂ", command=self.finish_crop, bg="#2ecc71", fg="white", font=("Arial", 10, "bold")).pack(side=tk.RIGHT, padx=5)
        
        tk.Label(btn_frame, text="(Ține click stânga apăsat pt contur)", fg="#bdc3c7", bg="#2c3e50").pack(side=tk.LEFT, padx=20)

        self.canvas = tk.Canvas(self.top, bg="gray", cursor="crosshair")
        self.canvas.pack(fill=tk.BOTH, expand=True)

        self.canvas.bind("<ButtonPress-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw_motion)
        self.canvas.bind("<ButtonRelease-1>", self.stop_draw)
        self.current_line = None
        self.img_offset_x = 0
        self.img_offset_y = 0

    def load_image(self):
        filename = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.jpeg *.png")])
        if filename:
            self.source_image = Image.open(filename).convert("RGBA")
            self.source_image.thumbnail((800, 600))
            self.tk_source = ImageTk.PhotoImage(self.source_image)
            self.canvas.delete("all")
            cx = self.canvas.winfo_width() // 2
            cy = self.canvas.winfo_height() // 2
            self.canvas.create_image(cx, cy, image=self.tk_source, anchor="center", tags="img_src")
            self.img_offset_x = cx - (self.source_image.width // 2)
            self.img_offset_y = cy - (self.source_image.height // 2)
            self.reset_selection()

    def start_draw(self, event):
        self.points = [(event.x, event.y)]
        self.current_line = self.canvas.create_line(event.x, event.y, event.x, event.y, fill="red", width=2, tags="lasso_line")

    def draw_motion(self, event):
        if self.points:
            self.points.append((event.x, event.y))
            coords = []
            for p in self.points: coords.extend(p)
            self.canvas.coords(self.current_line, *coords)

    def stop_draw(self, event):
        if len(self.points) > 2:
             self.canvas.create_line(self.points[-1][0], self.points[-1][1], self.points[0][0], self.points[0][1], fill="red", width=2, tags="lasso_line")

    def reset_selection(self):
        self.canvas.delete("lasso_line")
        self.points = []

    def finish_crop(self):
        if not self.source_image or len(self.points) < 3: return
        mask = Image.new("L", self.source_image.size, 0)
        draw_mask = ImageDraw.Draw(mask)
        real_points = []
        for px, py in self.points:
            real_points.append((px - self.img_offset_x, py - self.img_offset_y))
        draw_mask.polygon(real_points, fill=255, outline=255)
        result = Image.new("RGBA", self.source_image.size, (0,0,0,0))
        result.paste(self.source_image, (0,0), mask)
        bbox = mask.getbbox()
        if bbox: self.result_image = result.crop(bbox)
        else: self.result_image = result
        self.top.destroy()

# ==========================================
# CLASA PRINCIPALA - ROTATIE LIBERA
# ==========================================
class ForensicELAApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Proiect Criminalistică - ELA Expert")
        self.root.geometry("1300x800")

        # Variabile imagini
        self.base_image = None
        self.display_image = None
        self.tk_image = None
        self.bg_coords = (0, 0)

        # Variabile obiecte
        self.overlay_image = None         
        self.original_overlay_image = None 
        self.overlay_id = None
        self.drag_data = {"x": 0, "y": 0}
        self.interaction_mode = None
        self.text_to_place = ""
        self.current_scale = 1.0

        # --- GUI Setup Compact ---
        control_frame = tk.Frame(root, width=320, bg="#2c3e50", padx=10, pady=5)
        control_frame.pack(side=tk.LEFT, fill=tk.Y)
        
        tk.Label(control_frame, text="PANOU DE COMANDĂ", font=("Segoe UI", 11, "bold"), bg="#2c3e50", fg="white").pack(pady=(5, 10))

        # 1. GESTIUNE
        self.create_label(control_frame, "1. GESTIUNE FISIERE")
        row1 = tk.Frame(control_frame, bg="#2c3e50")
        row1.pack(fill=tk.X, pady=2)
        tk.Button(row1, text="📂 Încarcă", command=self.load_base_image, bg="#3498db", fg="white", width=12).pack(side=tk.LEFT, padx=2)
        tk.Button(row1, text="💾 Salvează", command=self.save_image, bg="#2980b9", fg="white", width=12).pack(side=tk.RIGHT, padx=2)
        tk.Button(control_frame, text="🔄 Reset la Original", command=self.reset_to_original, bg="grey", fg="white").pack(fill=tk.X, pady=5)
        
        self.lbl_size_comp = tk.Label(control_frame, text="Mărime: - KB", bg="#34495e", fg="#ecf0f1", font=("Arial", 9))
        self.lbl_size_comp.pack(fill=tk.X, pady=2)

        # 2. FALSIFICARE
        self.create_label(control_frame, "2. GENERATOR FALSURI")
        row2 = tk.Frame(control_frame, bg="#2c3e50")
        row2.pack(fill=tk.X, pady=2)
        tk.Button(row2, text="🅰️ Text", command=self.mode_add_text, bg="#e67e22", fg="white", width=12).pack(side=tk.LEFT, padx=2)
        tk.Button(row2, text="✂️ Lasso Tool", command=self.open_lasso_tool, bg="#d35400", fg="white", width=12).pack(side=tk.RIGHT, padx=2)
        
        # --- SLIDER ROTATIE NOU ---
        self.lbl_rot = tk.Label(control_frame, text="Rotire Obiect (Grade)", bg="#2c3e50", fg="white", font=("Arial", 9))
        self.lbl_rot.pack(pady=(5,0))
        # Slider de la 0 la 360 grade
        self.rotate_slider = Scale(control_frame, from_=0, to=360, orient=HORIZONTAL, bg="#2c3e50", fg="white", command=self.on_rotate_slide)
        self.rotate_slider.set(0)
        self.rotate_slider.pack(fill=tk.X)
        self.rotate_slider.config(state=tk.DISABLED)

        self.lbl_status = tk.Label(control_frame, text="Status: Așteptare...", bg="#2c3e50", fg="#f1c40f", font=("Arial", 8, "italic"), wraplength=280)
        self.lbl_status.pack(pady=2)

        # 3. COMPRESIE
        self.create_label(control_frame, "3. COMPRESIE JPEG")
        self.compress_slider = Scale(control_frame, from_=1, to=100, orient=HORIZONTAL, bg="#2c3e50", fg="white", label="Calitate (%)")
        self.compress_slider.set(70)
        self.compress_slider.pack(fill=tk.X)
        tk.Button(control_frame, text="📉 Aplică Compresia", command=self.apply_compression, bg="#c0392b", fg="white").pack(fill=tk.X, pady=2)

        # 4. ANALIZA ELA
        self.create_label(control_frame, "4. ANALIZA ELA")
        self.ela_scale = Scale(control_frame, from_=10, to=100, orient=HORIZONTAL, bg="#2c3e50", fg="white", label="Amplificare (Glow)")
        self.ela_scale.set(40)
        self.ela_scale.pack(fill=tk.X)
        
        self.ela_quality = Scale(control_frame, from_=50, to=100, orient=HORIZONTAL, bg="#2c3e50", fg="white", label="Calitate Referință")
        self.ela_quality.set(95)
        self.ela_quality.pack(fill=tk.X)
        
        tk.Button(control_frame, text="🔍 RULEAZĂ ELA", command=self.run_ela, bg="#9b59b6", fg="white", font=("Arial", 11, "bold"), height=2).pack(fill=tk.X, pady=10)

        # Canvas Main
        self.canvas = tk.Canvas(root, bg="#bdc3c7", cursor="cross")
        self.canvas.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH)
        
        # Binds
        self.canvas.bind("<Button-1>", self.on_canvas_click)
        self.canvas.bind("<B1-Motion>", self.on_canvas_drag)
        self.canvas.bind("<Button-3>", self.on_right_click)
        
        if platform.system() == "Linux":
            self.canvas.bind("<Button-4>", self.on_mouse_wheel)
            self.canvas.bind("<Button-5>", self.on_mouse_wheel)
        else:
            self.canvas.bind("<MouseWheel>", self.on_mouse_wheel)

    def create_label(self, p, t): 
        tk.Label(p, text=t, font=("Arial", 9,"bold"), bg="#34495e", fg="#ecf0f1").pack(fill=tk.X, pady=(10,2))

    def update_size_label(self):
        if self.display_image:
            buf = io.BytesIO()
            self.display_image.save(buf, "JPEG", quality=95)
            s = buf.tell() / 1024.0
            self.lbl_size_comp.config(text=f"Mărime: {s:.1f} KB")

    # --- FUNCTIE UNIFICATA DE TRANSFORMARE (ROTIRE + SCALARE) ---
    def apply_transformations(self):
        """Aplica atat rotatia curenta cat si scalarea curenta pe imaginea originala"""
        if self.original_overlay_image and self.overlay_id:
            # 1. Luam imaginea originala decupata
            img = self.original_overlay_image.copy()
            
            # 2. Aplicam rotirea (Rotim intai pentru a evita taierea colturilor la resize)
            angle = self.rotate_slider.get()
            if angle != 0:
                # expand=True este crucial ca sa nu taiem colturile
                img = img.rotate(angle, expand=True, resample=Image.BICUBIC)

            # 3. Aplicam scalarea
            if self.current_scale != 1.0:
                new_w = int(img.width * self.current_scale)
                new_h = int(img.height * self.current_scale)
                # Minim 1px
                new_w = max(1, new_w)
                new_h = max(1, new_h)
                img = img.resize((new_w, new_h), Image.LANCZOS)

            # 4. Actualizam imaginea pe ecran
            self.overlay_image = img # Salvam rezultatul transformat
            
            # Pastram centrul obiectului fix
            coords = self.canvas.coords(self.overlay_id)
            cx, cy = coords[0], coords[1]
            
            self.tk_overlay = ImageTk.PhotoImage(self.overlay_image)
            self.canvas.itemconfig(self.overlay_id, image=self.tk_overlay)
            self.canvas.coords(self.overlay_id, cx, cy)

    def on_rotate_slide(self, val):
        # Cand miscam sliderul, reaplicam transformarile
        if self.interaction_mode == "move_object":
            self.apply_transformations()

    def on_mouse_wheel(self, event):
        if self.interaction_mode == "move_object" and self.original_overlay_image:
            delta = 0
            if platform.system() == "Linux":
                if event.num == 4: delta = 1
                elif event.num == 5: delta = -1
            else:
                delta = event.delta

            if delta > 0: self.current_scale *= 1.1
            elif delta < 0: self.current_scale *= 0.9
            
            self.current_scale = max(0.1, min(self.current_scale, 3.0))
            
            self.lbl_status.config(text=f"Status: Scalare {(self.current_scale*100):.0f}%")
            # Apelam functia unificata
            self.apply_transformations()

    def open_lasso_tool(self):
        if not self.display_image:
            messagebox.showwarning("Atentie", "Incarca intai imaginea de baza!")
            return
        lasso_app = LassoSelectorApp(self.root)
        self.root.wait_window(lasso_app.top)
        if lasso_app.result_image:
            self.overlay_image = lasso_app.result_image
            self.place_overlay_on_canvas()
        else:
            self.lbl_status.config(text="Status: Selecție anulată.")

    def place_overlay_on_canvas(self):
        if self.overlay_image.width > 500 or self.overlay_image.height > 500:
             self.overlay_image.thumbnail((500, 500))
        self.original_overlay_image = self.overlay_image.copy()
        
        # Resetam parametrii
        self.current_scale = 1.0
        self.rotate_slider.set(0) # Resetam sliderul la 0
        
        self.tk_overlay = ImageTk.PhotoImage(self.overlay_image)
        cx = self.canvas.winfo_width() // 2
        cy = self.canvas.winfo_height() // 2
        self.overlay_id = self.canvas.create_image(cx, cy, image=self.tk_overlay, tags="overlay", anchor="center")
        self.interaction_mode = "move_object"
        
        # Activam sliderul
        self.rotate_slider.config(state=tk.NORMAL)
        self.lbl_status.config(text="Status: Obiect activ. Scroll pt zoom, Slider pt rotire.")

    def load_base_image(self):
        fn = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.png")])
        if fn:
            self.base_image = Image.open(fn).convert("RGB")
            self.base_image.thumbnail((1200, 900))
            self.display_image = self.base_image.copy()
            self.update_canvas(self.display_image)
            self.update_size_label()
            self.lbl_status.config(text="Status: Imagine încărcată.")
            self.interaction_mode = None
            self.rotate_slider.set(0)
            self.rotate_slider.config(state=tk.DISABLED)

    def update_canvas(self, pil_img):
        self.display_image = pil_img
        self.tk_image = ImageTk.PhotoImage(pil_img)
        self.canvas.delete("all")
        cx, cy = self.canvas.winfo_width(), self.canvas.winfo_height()
        if cx<10: cx=800
        if cy<10: cy=600
        ix, iy = max(0, (cx-pil_img.width)//2), max(0, (cy-pil_img.height)//2)
        self.canvas.create_image(ix, iy, image=self.tk_image, anchor="nw", tags="background")
        self.bg_coords = (ix, iy)

    def mode_add_text(self):
        if not self.display_image: return
        t = simpledialog.askstring("Text", "Text de inserat:")
        if t: self.text_to_place = t; self.interaction_mode = "text"; self.lbl_status.config(text="Status: Click pe imagine pt text.")

    def on_canvas_click(self, e):
        if self.interaction_mode == "text":
            rx, ry = int(e.x - self.bg_coords[0]), int(e.y - self.bg_coords[1])
            if 0 <= rx < self.display_image.width and 0 <= ry < self.display_image.height:
                draw = ImageDraw.Draw(self.display_image)
                try: font = ImageFont.truetype("arial.ttf", 50)
                except: font = ImageFont.load_default()
                draw.text((rx, ry), self.text_to_place, fill="red", font=font)
                self.update_canvas(self.display_image)
                self.interaction_mode = None
                self.update_size_label()
        elif self.interaction_mode == "move_object":
            item = self.canvas.find_closest(e.x, e.y)
            if "overlay" in self.canvas.gettags(item):
                self.drag_data["item"] = item; self.drag_data["x"] = e.x; self.drag_data["y"] = e.y

    def on_canvas_drag(self, e):
        if self.interaction_mode == "move_object" and "item" in self.drag_data:
            dx, dy = e.x - self.drag_data["x"], e.y - self.drag_data["y"]
            self.canvas.move(self.drag_data["item"], dx, dy)
            self.drag_data["x"], self.drag_data["y"] = e.x, e.y

    def on_right_click(self, e):
        if self.interaction_mode == "move_object" and self.overlay_image:
            ox, oy = self.canvas.coords(self.overlay_id)
            top_left_x = ox - (self.overlay_image.width / 2)
            top_left_y = oy - (self.overlay_image.height / 2)
            paste_x = int(top_left_x - self.bg_coords[0])
            paste_y = int(top_left_y - self.bg_coords[1])
            
            # Lipire cu masca de transparenta
            if self.overlay_image.mode == 'RGBA':
                self.display_image.paste(self.overlay_image, (paste_x, paste_y), self.overlay_image)
            else:
                self.display_image.paste(self.overlay_image, (paste_x, paste_y))

            self.canvas.delete(self.overlay_id)
            self.overlay_id = None; self.overlay_image = None; self.original_overlay_image = None
            self.interaction_mode = None
            self.rotate_slider.config(state=tk.DISABLED)
            self.update_canvas(self.display_image)
            self.update_size_label()
            self.lbl_status.config(text="Status: Obiect fixat.")

    def apply_compression(self):
        if not self.display_image: return
        q = self.compress_slider.get()
        buf = io.BytesIO()
        self.display_image.save(buf, "JPEG", quality=q)
        s = len(buf.getvalue())/1024.0
        self.lbl_size_comp.config(text=f"Mărime: {s:.1f} KB")
        buf.seek(0)
        self.display_image = Image.open(buf).convert("RGB")
        self.update_canvas(self.display_image)
        messagebox.showinfo("Compresie", f"Comprimat la Q={q}. Mărime: {s:.1f} KB")

    def run_ela(self):
        if not self.display_image: return
        orig = self.display_image
        buf = io.BytesIO()
        orig.save(buf, "JPEG", quality=self.ela_quality.get())
        buf.seek(0)
        recomp = Image.open(buf).convert("RGB")
        ela = ImageChops.difference(orig, recomp)
        ela = ImageEnhance.Brightness(ela).enhance(self.ela_scale.get())
        top = Toplevel(self.root); top.title("Rezultat ELA"); top.geometry("1000x800")
        tk_ela = ImageTk.PhotoImage(ela); lbl = tk.Label(top, image=tk_ela); lbl.image = tk_ela; lbl.pack(expand=True)

    def reset_to_original(self):
        if self.base_image:
            self.display_image = self.base_image.copy()
            self.update_canvas(self.display_image)
            self.canvas.delete("overlay")
            self.rotate_slider.config(state=tk.DISABLED)
            self.update_size_label()

    def save_image(self):
        if self.display_image:
            f = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG", "*.jpg")])
            if f: self.display_image.save(f, quality=95)

if __name__ == "__main__":
    root = tk.Tk()
    app = ForensicELAApp(root)
    root.mainloop()